In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2
import pickle
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Read in the locations of the parts that are labeled on each image, and join them with the address of the corresponding images. This is to be able to train a classifier to distinguish between each `part_id` by looking at that part in the image.

In [2]:
NUM_CLASSES = 20 # limit classes trained

import matplotlib.patches as patches
from collections import defaultdict
from skimage import io, transform
from numpy.random import choice

# GENERATE THE TRAINING DATA

# CLASS_IMGS = defaultdict(list)
# first level key = class_id. second level key = part_id

class_ids = set()

CLASS_INSTANCES = []
IMG_LABELS = []
TEST_INSTANCES = []
TEST_LABELS = []
# prev_class_id = 0
train_test_ratio = [0.9, 0.1]

im_addr_classes = pickle.load(open('metadata', 'rb'))
for index, row in im_addr_classes.iterrows():
    im_id, im_addr, bb_x, bb_y, bb_w, bb_h, class_id = row
    
    if class_id not in class_ids:
        if len(class_ids) >= NUM_CLASSES:
            break
        class_ids.add(class_id)
    
    img = io.imread('CUB_200_2011/images/' + im_addr)
    
    bb_x, bb_y, bb_w, bb_h = [int(x) for x in [bb_x, bb_y, bb_w, bb_h]]
    if len(img.shape) == 2:
        img = np.dstack([img, img, img])
    if 0 in img.shape:
        continue

    img = img[bb_y:bb_y + bb_h, bb_x:bb_x + bb_w, :]
    img = transform.resize(img, (224, 224, 3))
#     plt.imshow(img)
#     plt.show()
        

#     print('extracting foreground for', class_id, im_addr)
    if choice([True, False], p=train_test_ratio):
        CLASS_INSTANCES.append(img)
        IMG_LABELS.append(class_id)
    else:
        TEST_INSTANCES.append(img)
        TEST_LABELS.append(class_id)

print(np.array(CLASS_INSTANCES).shape)
print(np.array(TEST_INSTANCES).shape)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(988, 224, 224, 3)
(127, 224, 224, 3)


In [3]:
all_imgs_processed = np.array(CLASS_INSTANCES)
all_labels = np.array(IMG_LABELS)
# np.savez_compressed('images_normalized', all_imgs_processed)
# np.savez_compressed('images_normalize_labels', all_labels)

In [4]:
# VGGNet training

# all_imgs_processed = np.load('images_normalized.npz')
# all_labels = np.load('images_normalized_labels.npz')
import vgg as vgg_mod
import importlib
importlib.reload(vgg_mod)

sess = tf.Session()

vgg_obj = None
# graph = tf.Graph()
# with graph.as_default():
imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
vgg_obj = vgg_mod.vgg16(imgs, 'vgg16_weights.npz', sess)

labels_ph = tf.placeholder(tf.int64, (None), name='labels_ph')
labels_ph_oh = tf.one_hot(labels_ph, 20)
#     print(labels_ph_oh)
#     vgg_obj.load_weights(vgg_obj.weights, vgg_obj.sess)

# print(tf.trainable_variables())

# Get trainable variables for the last three custom FC layers (featuring dropout, etc)
fc_stem = 'fully_connected'
trainable_vars_list = []
for i in ['', '_1', '_2']:
    trainable_weight_var = tf.contrib.framework.get_variables(fc_stem + i + '/weights:0')
    trainable_bias_var = tf.contrib.framework.get_variables(fc_stem + i + '/biases:0')
    trainable_vars_list += [trainable_weight_var, trainable_bias_var]

optimizer_warm = tf.train.AdamOptimizer(0.001, 0.9, 0.999)
loss_warm = tf.nn.sigmoid_cross_entropy_with_logits(logits=vgg_obj.logits, labels=labels_ph_oh)
opt_warm = optimizer_warm.minimize(loss_warm, var_list=trainable_vars_list)

optimizer_full = tf.train.AdamOptimizer(0.00001, 0.9, 0.999)
loss_full = tf.nn.sigmoid_cross_entropy_with_logits(logits=vgg_obj.logits, labels=labels_ph_oh)
opt_full = optimizer_full.minimize(loss_full)

correct = tf.equal(tf.argmax(vgg_obj.probs, 1), labels_ph)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        
# tf.get_default_graph().finalize()
sess.run(tf.global_variables_initializer())
vgg_obj.load_weights(vgg_obj.weights, vgg_obj.sess)



0 conv1_1_W (3, 3, 3, 64)
1 conv1_1_b (64,)
2 conv1_2_W (3, 3, 64, 64)
3 conv1_2_b (64,)
4 conv2_1_W (3, 3, 64, 128)
5 conv2_1_b (128,)
6 conv2_2_W (3, 3, 128, 128)
7 conv2_2_b (128,)
8 conv3_1_W (3, 3, 128, 256)
9 conv3_1_b (256,)
10 conv3_2_W (3, 3, 256, 256)
11 conv3_2_b (256,)
12 conv3_3_W (3, 3, 256, 256)
13 conv3_3_b (256,)
14 conv4_1_W (3, 3, 256, 512)
15 conv4_1_b (512,)
16 conv4_2_W (3, 3, 512, 512)
17 conv4_2_b (512,)
18 conv4_3_W (3, 3, 512, 512)
19 conv4_3_b (512,)
20 conv5_1_W (3, 3, 512, 512)
21 conv5_1_b (512,)
22 conv5_2_W (3, 3, 512, 512)
23 conv5_2_b (512,)
24 conv5_3_W (3, 3, 512, 512)
25 conv5_3_b (512,)


In [5]:
# Training code
BS = 32
NUM_EPOCHS_WARM = 20
NUM_EPOCHS_FULL = 20

image_data = all_imgs_processed
label_data = all_labels

training = vgg_obj.training
keep_prob = vgg_obj.keep_prob

def train_for_epochs(optimizer, loss, epochs):
    for epoch in range(epochs):
        # Let's shuffle the data every epoch
        np.random.seed(epoch)
        np.random.shuffle(image_data)
        np.random.seed(epoch)
        np.random.shuffle(label_data)
        # Go through the entire dataset once
        accuracy_vals, loss_vals = [], []
        for i in range(0, image_data.shape[0]-BS+1, BS):
            # Train a single batch
            batch_images, batch_labels = image_data[i:i+BS], label_data[i:i+BS]
            accuracy_val, loss_val, _ = sess.run([accuracy, loss, optimizer], feed_dict={imgs: batch_images, labels_ph: batch_labels, training: True, keep_prob: 0.9})
            accuracy_vals.append(accuracy_val)
            loss_vals.append(loss_val)

    #     val_correct = []
    #     for i in range(0, image_val.shape[0], BS):
    #         batch_images, batch_labels = image_val[i:i+BS], label_val[i:i+BS]
    #         val_correct.extend( sess.run(correct, feed_dict={eval_inputs: batch_images, labels_ph: batch_labels}) )
        print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f '%(epoch, np.mean(accuracy_vals), np.mean(loss_vals)))
train_for_epochs(opt_warm, loss_warm, NUM_EPOCHS_WARM)
train_for_epochs(opt_full, loss_full, NUM_EPOCHS_FULL)

[  0] Accuracy: 0.078  	  Loss: 0.796 
[  1] Accuracy: 0.159  	  Loss: 0.766 
[  2] Accuracy: 0.235  	  Loss: 0.747 
[  3] Accuracy: 0.267  	  Loss: 0.737 
[  4] Accuracy: 0.341  	  Loss: 0.725 
[  5] Accuracy: 0.354  	  Loss: 0.718 
[  6] Accuracy: 0.401  	  Loss: 0.716 
[  7] Accuracy: 0.443  	  Loss: 0.708 
[  8] Accuracy: 0.438  	  Loss: 0.704 
[  9] Accuracy: 0.494  	  Loss: 0.693 
[ 10] Accuracy: 0.488  	  Loss: 0.693 
[ 11] Accuracy: 0.509  	  Loss: 0.685 
[ 12] Accuracy: 0.551  	  Loss: 0.685 
[ 13] Accuracy: 0.548  	  Loss: 0.685 
[ 14] Accuracy: 0.551  	  Loss: 0.679 
[ 15] Accuracy: 0.545  	  Loss: 0.675 
[ 16] Accuracy: 0.564  	  Loss: 0.670 
[ 17] Accuracy: 0.562  	  Loss: 0.669 
[ 18] Accuracy: 0.594  	  Loss: 0.663 
[ 19] Accuracy: 0.591  	  Loss: 0.664 
[  0] Accuracy: 0.524  	  Loss: 0.682 
[  1] Accuracy: 0.566  	  Loss: 0.672 
[  2] Accuracy: 0.575  	  Loss: 0.669 
[  3] Accuracy: 0.607  	  Loss: 0.667 
[  4] Accuracy: 0.605  	  Loss: 0.664 
[  5] Accuracy: 0.610  	 

In [9]:
test_imgs_processed = np.array(TEST_INSTANCES)
test_labels = np.array(TEST_LABELS)
test_probabilities = sess.run(vgg_obj.probs, feed_dict={vgg_obj.imgs: test_imgs_processed, training: False, keep_prob:0.9})

top_1_correct, top_3_correct, top_5_correct = [0] * 3
for sample_idx, sample in enumerate(test_probabilities):
    top_predictions = (np.argsort(sample)[::-1])[:5]
    test_label = test_labels[sample_idx]
    if test_label in top_predictions[:1]:
        top_1_correct += 1
    if test_label in top_predictions[:3]:
        top_3_correct += 1
    if test_label in top_predictions:
        top_5_correct += 1

print('Test accuracy:')
print('Top-1:', top_1_correct / len(test_imgs_processed))
print('Top-3:', top_3_correct / len(test_imgs_processed))
print('Top-5:', top_5_correct / len(test_imgs_processed))

Test accuracy:
Top-1: 0.06299212598425197
Top-3: 0.16535433070866143
Top-5: 0.2677165354330709


In [14]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, './checkpoints/bird_657_acc.ckpt')

'./checkpoints/bird_657_acc.ckpt'